In [1]:
# Импорт всех необходимых библиотек
import os
import sys
import subprocess
import time
import shutil
import warnings
warnings.filterwarnings('ignore')

# Для работы с путями и файлами
from pathlib import Path

# Для работы с данными
import numpy as np
import pandas as pd

# Для работы с изображениями
import cv2
from PIL import Image

# Для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

# Для работы с конфигами YAML
import yaml

# Для отображения в ноутбуке
from IPython.display import display, Image as IPImage, Markdown, HTML

# PyTorch (основа YOLOv5)
import torch
import torchvision

print("=" * 60)
print("ВСЕ БИБЛИОТЕКИ ЗАГРУЖЕНЫ")
print("=" * 60)

# Вывод версий
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Torchvision: {torchvision.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"NumPy: {np.__version__}")

# Проверка GPU
if torch.cuda.is_available():
    print(f"\nGPU доступна: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA версия: {torch.version.cuda}")
else:
    print("\nGPU не найдена, будет использоваться CPU")
    print("   Обучение будет медленнее")

print("\nВсе библиотеки готовы к работе!")

ВСЕ БИБЛИОТЕКИ ЗАГРУЖЕНЫ
Python: 3.14.2 (tags/v3.14.2:df79316, Dec  5 2025, 17:18:21) [MSC v.1944 64 bit (AMD64)]
PyTorch: 2.9.1+cpu
Torchvision: 0.24.1+cpu
OpenCV: 4.12.0
NumPy: 2.2.6

GPU не найдена, будет использоваться CPU
   Обучение будет медленнее

Все библиотеки готовы к работе!


In [2]:
# Настройка проекта YOLOv5 для распознавания человека
print("Проект: Распознавание конкретного человека с YOLOv5")
print("=" * 50)

# Запуск создания структуры проекта
!python create_structure.py

Проект: Распознавание конкретного человека с YOLOv5
Создание структуры проекта для YOLOv5

Создаю папки...
  Создано: datasets
  Создано: datasets/images/train
  Создано: datasets/images/val
  Создано: datasets/images/test
  Создано: datasets/labels/train
  Создано: datasets/labels/val
  Создано: datasets/labels/test
  Создано: models
  Создано: test_videos
  Создано: results
  Создано: scripts
  Создано: README.md

Структура проекта создана!

Теперь соберите фотографии человека и разместите их:
  Фотографии: datasets/images/train/ и datasets/images/val/
  Разметка:   datasets/labels/train/ и datasets/labels/val/


In [3]:
# Скачивание YOLOv5
!python scripts/download_yolo.py

Скачивание YOLOv5

Проверяю виртуальное окружение...
Вирутальное окружение активировано

Скачиваю YOLOv5...
Клонирую репозиторий...
YOLOv5 успешно скачан

Устанавливаю зависимости YOLOv5...
Зависимости YOLOv5 установлены

Готово!


In [4]:
# Исправление YOLOv5 для новой версии PyTorch
!python scripts/fix_yolov5_amp.py

ИСПРАВЛЕНИЕ YOLOv5 ДЛЯ НОВОЙ ВЕРСИИ PyTorch
Создание резервных копий...
✅ Резервная копия создана: yolov5_backup\yolov5_backup_20251224_195253
Исправление AMP API в YOLOv5...
Исправляю yolov5\train.py...
✅ Исправлен torch.cuda.amp.autocast в train.py

Проверяю другие файлы...

✅ Исправления применены!
Перезапустите обучение.

📋 Инструкция:
1. Запустите обучение заново
2. Если будут новые ошибки - можно восстановить из backup:
   restore_from_backup.py yolov5_backup_20251224_195253


In [5]:
# Проверка что все установлено
!python scripts/test_setup.py

Проверка установки

1. Python версия:
   3.14.2 (tags/v3.14.2:df79316, Dec  5 2025, 17:18:21) [MSC v.1944 64 bit (AMD64)]

2. PyTorch:
   Версия: 2.9.1+cpu
   GPU не найдена, будет использоваться CPU

3. OpenCV:
   Версия: 4.12.0

4. Создание тестового изображения:
   Изображение создано: test_image.jpg
   Размер: 640x480 пикселей

Проверка завершена!
Все установлено, но GPU не найдена.
Обучение будет медленнее на CPU.


In [6]:
# Анализ собранных данных
!python scripts/run_analysis.py

Проверка загруженных данных...
Обучающие изображения: 93
Валидационные изображения: 20
Обучающие разметки: 93
Валидационные разметки: 20

Примеры изображений:
Figure(1500x500)
Проверка конфигурационного файла...
✅ data.yaml от Roboflow найден
Содержимое data.yaml:
  Классы: 1
  Имена классов: ['niki']
  Путь к train: images/train
  Путь к val: images/val
✅ Пути исправлены


In [7]:
# Тестируем ресайз YOLO
import cv2
import numpy as np
from pathlib import Path

def test_yolo_resize(image_path, target_size=640):
    """Показывает как YOLO изменит фото"""
    
    img = cv2.imread(str(image_path))
    if img is None:
        return None
    
    h, w = img.shape[:2]
    print(f"Оригинал: {w}x{h}")
    
    # Метод YOLO с --rect (letterbox)
    scale = min(target_size / w, target_size / h)
    new_w = int(w * scale)
    new_h = int(h * scale)
    
    resized = cv2.resize(img, (new_w, new_h))
    
    # Добавляем чёрные полосы
    result = np.full((target_size, target_size, 3), 114, dtype=np.uint8)
    dx = (target_size - new_w) // 2
    dy = (target_size - new_h) // 2
    result[dy:dy+new_h, dx:dx+new_w] = resized
    
    return result

# Протестируйте на одном фото
test_image = list(Path("datasets/images/train").glob("*.*"))[0]
result = test_yolo_resize(test_image, 640)

if result is not None:
    print(f"\nПосле YOLO --rect: 640x640")
    print(f"Чёрные полосы по бокам, но пропорции сохранены!")
    
    # Сохраните для просмотра
    cv2.imwrite("test_yolo_resize.jpg", result)
    print("Сохранено: test_yolo_resize.jpg")

Оригинал: 1020x680

После YOLO --rect: 640x640
Чёрные полосы по бокам, но пропорции сохранены!
Сохранено: test_yolo_resize.jpg


In [16]:
# Обучение модели (когда будут данные)
!python scripts/run_train.py

Начинаю обучение модели...
Оптимизировано для CPU 16GB
БЕЗ mosaic - для точного распознавания

Параметры обучения:
  Batch size: 8 (было 4)
  Epochs: 200 (было 150)
  Модель: yolov5m (было yolov5s) - лучше точность
  Patience: 50 (было 10)
  Multi-scale: ✅ ВКЛЮЧЕНО (безопасно)
  Файл аугментаций: data/hyps/hyp.custom.yaml
  Mosaic: ❌ ОТКЛЮЧЕНО (в hyp.custom.yaml)
  Другие аугментации: умеренные (безопасные)

Начинаю обучение...

Последние строки вывода:

✅ Обучение завершено за 326.0 минут
Модель сохранена: results/training_run/weights/best.pt


In [26]:
# Итог обучения
!python scripts/check_progress.py

                    ИТОГОВЫЙ ОТЧЕТ ОБ ОБУЧЕНИИ
Папка результатов: training_run

📊 Файл результатов: results.csv
📈 Всего эпох: 197

📋 ФИНАЛЬНЫЕ МЕТРИКИ (последняя эпоха):
  Точность (precision): 0.783
  Полнота (recall): 0.636
  mAP@0.5: 0.734
  mAP@0.5:0.95: 0.485

🏆 ЛУЧШИЕ МЕТРИКИ (за все обучение):
  Точность (precision): 0.933 (эпоха 178)
  Полнота (recall): 0.955 (эпоха 4)
  mAP@0.5: 0.784 (эпоха 101)
  mAP@0.5:0.95: 0.541 (эпоха 147)

📊 СТАТИСТИКА ПО ОБУЧЕНИЮ:
  mAP@0.5:
    Лучший: 0.784
    Худший: 0.010
    Средний: 0.616
    Финальный: 0.734

  Train Loss:
    Начальный: 0.078
    Финальный: 0.014
    Улучшение: 82.1%

📊 ГРАФИКИ ОБУЧЕНИЯ:

📊 График Precision/Recall:
----------------------------------------------------------------------
                             Precision/Recall по эпохам                         
    ┌──────────────────────────────────────────────────────────────────────────┐
0.95┤ •• Precision          •                          •     •      •••••      │
  

In [ ]:
# Тестирование модели (после обучения)

!python scripts/test_on_test_folder.py

Для тестирования выполните:
!cd yolov5 && python detect.py --weights runs/train/exp/weights/best.pt --source test_image.jpg
Тестирование модели YOLOv5
----------------------------------------
Модель: results/training_run/weights/best.pt
Найдено фото для теста: 23

Запускаю тестирование...
Команда: d:\Vika\part_2\hw3\venv\Scripts\python.exe detect.py --weights ../results/training_run/weights/best.pt --source ../datasets/images/test --conf 0.5 --project ../results --name final_test --exist-ok

✅ Тестирование завершено!
Результаты сохранены в: results/final_test/

Обработанные файлы (23):
  - 063dc2ba4a9e8ffbeb01eaf2ebdb23b0.jpg
  - 09a6d3c3b787cc424ba957fc0246293c.jpg
  - 35737d1a1494845454f318c664a3d593.jpg
  - 3m7tu3a3pnq71.jpg
  - 428ff76b6a28123c18e4694c0d38b485.jpg
  - 6ce3580d1e546285aeea074750ac74a1.jpg
  - 83a8a0d07f06f6fefa8322283921ee1f.jpg
  - 9bdf8fdb31f4f8573bb3a2a51f30fcea.jpg
  - a9a6cc94fb47466a959ae33586d50395.jpg
  - aa3ccfce3c4a4ab7a1cd0ce9e1bd957f642.jpg
  ... и ещё 1

detect: weights=['../results/training_run/weights/best.pt'], source=../datasets/images/test, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../results, name=final_test, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-453-geed9bc19 Python-3.14.2 torch-2.9.1+cpu CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/23 D:\Vika\part_2\hw3\datasets\images\test\063dc2ba4a9e8ffbeb01eaf2ebdb23b0.jpg: 640x640 (no detections), 460.0ms
image 2/23 D:\Vika\part_2\hw3\datasets\images\test\09a6d3c3b787cc424ba957fc0246293c.jpg: 640x640 1 niki, 442.0ms
image 3/23 D:\Vika\part_2\hw3\datasets\images\test\35737d1a1494845454f318c664a3d593.jpg: 

In [ ]:
# Обработка видео
print("Для обработки видео:")
print("!cd yolov5 && python detect.py --weights models/best.pt --source test_videos/ваше_видео.mp4")

Для обработки видео:
!cd yolov5 && python detect.py --weights models/best.pt --source test_videos/ваше_видео.mp4
